### Chatbot

In [1]:
import random 

import joblib

import sys
import os


import random
random.seed(42)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, Dropout
from tensorflow.keras.models import Model

Using TensorFlow backend.
C:\Users\Usuario\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Usuario\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Usuario\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Usuario\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
df = pd.read_excel(r"database_intents.xlsx")

In [3]:
# Usar get_dummies en vez de las 3 funciones
df_oh = pd.concat([df, pd.get_dummies(df['Intent type'])], axis = 1)

In [4]:
df_train, df_test = train_test_split(df_oh, train_size = 0.7, test_size = 0.3, random_state = 42,
                                    shuffle = True, stratify = df_oh['Intent type'])

In [5]:
stopwords = []

In [6]:
cv = TfidfVectorizer(
    stop_words= stopwords,
    ngram_range=(1, 4),
    strip_accents='ascii',
    max_df=0.99,
    min_df=0,
    max_features=100
    )
cv.fit(df_train["Sentence"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.99, max_features=100, min_df=0,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=[], strip_accents='ascii', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [9]:
import pickle

with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(cv, file)

In [7]:
def processing(df, pretreatment = False, Tfidf = True, cv = None, stopwords = []):
  # Normalizamos y limpiamos el corpus 
  if pretreatment == True:
    df["Sentence"] = df['Sentence'].apply(lambda x: word_treatment(x))
    print("El corpus ha sido pretratado")
    
     # Transformamos nuestro corpus a un vector Tfidf
  if Tfidf == True:

    if cv == None:
      cv = TfidfVectorizer(
        stop_words= stopwords,
        ngram_range=(1, 4),
        strip_accents='ascii',
        max_df=0.99,
        min_df=0,
        max_features=100
      )
      cv.fit(df["Sentence"])
      X = cv.transform(df["Sentence"])
      print("Se ha realizado una vectorización Tfidf")
      return df, X, cv

    else:
      X = cv.transform(df["Sentence"])
      #print("Se ha realizado una vectorización Tfidf basado en el corpus suministrado por cv")
    return df, X
  else:
    return df


In [8]:
df_train, X_train, cv = processing(df_train)
df_test, X_test = processing(df_test, cv = cv)

Se ha realizado una vectorización Tfidf


In [9]:
y_train = df_train[["Greeting","Search","Suggestions"]]
y_test = df_test[["Greeting","Search","Suggestions"]]

In [10]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.7, 
test_size=0.3, random_state=42, shuffle=True, stratify=df_train["Intent type"])

In [11]:
shape= X_train.shape[1]

X_train.sort_indices()
X_validation.sort_indices()

In [12]:
def mlp_greeting(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation = "sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [13]:
mlp_greeting = mlp_greeting(shape)
history = mlp_greeting.fit(X_train, np.asarray(y_train["Greeting"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Greeting"]).reshape(-1,1)),
    epochs=200,
    workers = 2, use_multiprocessing= True, verbose = 2)

Train on 58 samples, validate on 26 samples
Epoch 1/200
 - 4s - loss: 0.6913 - acc: 0.5517 - val_loss: 0.6899 - val_acc: 0.5769
Epoch 2/200
 - 0s - loss: 0.6893 - acc: 0.4655 - val_loss: 0.6864 - val_acc: 0.6154
Epoch 3/200
 - 0s - loss: 0.6851 - acc: 0.6207 - val_loss: 0.6830 - val_acc: 0.6923
Epoch 4/200
 - 0s - loss: 0.6771 - acc: 0.7241 - val_loss: 0.6794 - val_acc: 0.7308
Epoch 5/200
 - 0s - loss: 0.6778 - acc: 0.5690 - val_loss: 0.6759 - val_acc: 0.7308
Epoch 6/200
 - 0s - loss: 0.6793 - acc: 0.6552 - val_loss: 0.6723 - val_acc: 0.7308
Epoch 7/200
 - 0s - loss: 0.6733 - acc: 0.7414 - val_loss: 0.6688 - val_acc: 0.7308
Epoch 8/200
 - 0s - loss: 0.6731 - acc: 0.7069 - val_loss: 0.6653 - val_acc: 0.7308
Epoch 9/200
 - 0s - loss: 0.6769 - acc: 0.7241 - val_loss: 0.6618 - val_acc: 0.7308
Epoch 10/200
 - 0s - loss: 0.6859 - acc: 0.7241 - val_loss: 0.6583 - val_acc: 0.7308
Epoch 11/200
 - 0s - loss: 0.6600 - acc: 0.7414 - val_loss: 0.6549 - val_acc: 0.7308
Epoch 12/200
 - 0s - loss: 0.6

Epoch 97/200
 - 0s - loss: 0.4279 - acc: 0.7586 - val_loss: 0.3951 - val_acc: 0.7308
Epoch 98/200
 - 0s - loss: 0.4278 - acc: 0.7586 - val_loss: 0.3935 - val_acc: 0.7308
Epoch 99/200
 - 0s - loss: 0.4057 - acc: 0.7586 - val_loss: 0.3919 - val_acc: 0.7308
Epoch 100/200
 - 0s - loss: 0.4097 - acc: 0.7586 - val_loss: 0.3904 - val_acc: 0.7308
Epoch 101/200
 - 0s - loss: 0.4071 - acc: 0.7586 - val_loss: 0.3889 - val_acc: 0.7308
Epoch 102/200
 - 0s - loss: 0.3503 - acc: 0.7586 - val_loss: 0.3873 - val_acc: 0.7308
Epoch 103/200
 - 0s - loss: 0.3668 - acc: 0.7586 - val_loss: 0.3858 - val_acc: 0.7308
Epoch 104/200
 - 0s - loss: 0.3897 - acc: 0.7586 - val_loss: 0.3841 - val_acc: 0.7308
Epoch 105/200
 - 0s - loss: 0.3703 - acc: 0.7586 - val_loss: 0.3825 - val_acc: 0.7308
Epoch 106/200
 - 0s - loss: 0.3432 - acc: 0.7586 - val_loss: 0.3810 - val_acc: 0.7308
Epoch 107/200
 - 0s - loss: 0.3487 - acc: 0.7586 - val_loss: 0.3795 - val_acc: 0.7308
Epoch 108/200
 - 0s - loss: 0.4168 - acc: 0.7586 - val_lo

Epoch 193/200
 - 0s - loss: 0.2154 - acc: 0.8966 - val_loss: 0.3993 - val_acc: 0.8846
Epoch 194/200
 - 0s - loss: 0.2193 - acc: 0.8966 - val_loss: 0.4000 - val_acc: 0.8846
Epoch 195/200
 - 0s - loss: 0.2245 - acc: 0.9138 - val_loss: 0.4007 - val_acc: 0.8846
Epoch 196/200
 - 0s - loss: 0.2346 - acc: 0.9138 - val_loss: 0.4016 - val_acc: 0.8846
Epoch 197/200
 - 0s - loss: 0.2421 - acc: 0.9138 - val_loss: 0.4028 - val_acc: 0.8846
Epoch 198/200
 - 0s - loss: 0.2234 - acc: 0.9483 - val_loss: 0.4049 - val_acc: 0.8846
Epoch 199/200
 - 0s - loss: 0.2032 - acc: 0.9310 - val_loss: 0.4067 - val_acc: 0.8846
Epoch 200/200
 - 0s - loss: 0.2229 - acc: 0.9483 - val_loss: 0.4075 - val_acc: 0.8846


In [14]:
def mlp_search(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation="sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [15]:
mlp_search = mlp_search(shape)
history = mlp_search.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs=250,
    workers = 2, use_multiprocessing= True, verbose = 2)

Train on 58 samples, validate on 26 samples
Epoch 1/250
 - 1s - loss: 0.6952 - acc: 0.6207 - val_loss: 0.6927 - val_acc: 0.7692
Epoch 2/250
 - 0s - loss: 0.6958 - acc: 0.6897 - val_loss: 0.6923 - val_acc: 0.7692
Epoch 3/250
 - 0s - loss: 0.6937 - acc: 0.6897 - val_loss: 0.6917 - val_acc: 0.7692
Epoch 4/250
 - 0s - loss: 0.6916 - acc: 0.7586 - val_loss: 0.6911 - val_acc: 0.7692
Epoch 5/250
 - 0s - loss: 0.6910 - acc: 0.7586 - val_loss: 0.6905 - val_acc: 0.7692
Epoch 6/250
 - 0s - loss: 0.6913 - acc: 0.7241 - val_loss: 0.6899 - val_acc: 0.7692
Epoch 7/250
 - 0s - loss: 0.6912 - acc: 0.7069 - val_loss: 0.6893 - val_acc: 0.7692
Epoch 8/250
 - 0s - loss: 0.6898 - acc: 0.7414 - val_loss: 0.6886 - val_acc: 0.7692
Epoch 9/250
 - 0s - loss: 0.6902 - acc: 0.7241 - val_loss: 0.6880 - val_acc: 0.7692
Epoch 10/250
 - 0s - loss: 0.6881 - acc: 0.7586 - val_loss: 0.6873 - val_acc: 0.7692
Epoch 11/250
 - 0s - loss: 0.6872 - acc: 0.7586 - val_loss: 0.6866 - val_acc: 0.7692
Epoch 12/250
 - 0s - loss: 0.6

Epoch 97/250
 - 0s - loss: 0.5800 - acc: 0.7586 - val_loss: 0.5726 - val_acc: 0.7692
Epoch 98/250
 - 0s - loss: 0.5680 - acc: 0.7586 - val_loss: 0.5703 - val_acc: 0.7692
Epoch 99/250
 - 0s - loss: 0.5695 - acc: 0.7586 - val_loss: 0.5682 - val_acc: 0.7692
Epoch 100/250
 - 0s - loss: 0.5723 - acc: 0.7586 - val_loss: 0.5660 - val_acc: 0.7692
Epoch 101/250
 - 0s - loss: 0.5562 - acc: 0.7586 - val_loss: 0.5638 - val_acc: 0.7692
Epoch 102/250
 - 0s - loss: 0.5471 - acc: 0.7586 - val_loss: 0.5616 - val_acc: 0.7692
Epoch 103/250
 - 0s - loss: 0.5676 - acc: 0.7586 - val_loss: 0.5593 - val_acc: 0.7692
Epoch 104/250
 - 0s - loss: 0.5548 - acc: 0.7586 - val_loss: 0.5571 - val_acc: 0.7692
Epoch 105/250
 - 0s - loss: 0.5382 - acc: 0.7586 - val_loss: 0.5549 - val_acc: 0.7692
Epoch 106/250
 - 0s - loss: 0.5417 - acc: 0.7586 - val_loss: 0.5527 - val_acc: 0.7692
Epoch 107/250
 - 0s - loss: 0.5617 - acc: 0.7586 - val_loss: 0.5507 - val_acc: 0.7692
Epoch 108/250
 - 0s - loss: 0.5499 - acc: 0.7586 - val_lo

Epoch 193/250
 - 0s - loss: 0.3777 - acc: 0.7586 - val_loss: 0.4164 - val_acc: 0.7692
Epoch 194/250
 - 0s - loss: 0.4376 - acc: 0.7586 - val_loss: 0.4159 - val_acc: 0.7692
Epoch 195/250
 - 0s - loss: 0.3872 - acc: 0.7586 - val_loss: 0.4153 - val_acc: 0.7692
Epoch 196/250
 - 0s - loss: 0.4069 - acc: 0.7586 - val_loss: 0.4145 - val_acc: 0.7692
Epoch 197/250
 - 0s - loss: 0.4262 - acc: 0.7586 - val_loss: 0.4134 - val_acc: 0.7692
Epoch 198/250
 - 0s - loss: 0.3640 - acc: 0.7586 - val_loss: 0.4128 - val_acc: 0.7692
Epoch 199/250
 - 0s - loss: 0.3974 - acc: 0.7586 - val_loss: 0.4125 - val_acc: 0.7692
Epoch 200/250
 - 0s - loss: 0.4538 - acc: 0.7586 - val_loss: 0.4117 - val_acc: 0.7692
Epoch 201/250
 - 0s - loss: 0.4347 - acc: 0.7586 - val_loss: 0.4106 - val_acc: 0.7692
Epoch 202/250
 - 0s - loss: 0.3908 - acc: 0.7586 - val_loss: 0.4092 - val_acc: 0.7692
Epoch 203/250
 - 0s - loss: 0.4145 - acc: 0.7586 - val_loss: 0.4077 - val_acc: 0.7692
Epoch 204/250
 - 0s - loss: 0.4041 - acc: 0.7586 - val

In [16]:
def mlp_suggestion(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation="sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer = opt)
# return our model
    return model

In [17]:
mlp_suggestion = mlp_suggestion(shape)
history = mlp_suggestion.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs=250,
    workers = 2, use_multiprocessing= True, verbose = 2)

Train on 58 samples, validate on 26 samples
Epoch 1/250
 - 1s - loss: 0.6945 - acc: 0.5517 - val_loss: 0.6888 - val_acc: 0.6923
Epoch 2/250
 - 0s - loss: 0.6931 - acc: 0.5517 - val_loss: 0.6870 - val_acc: 0.7308
Epoch 3/250
 - 0s - loss: 0.6838 - acc: 0.6897 - val_loss: 0.6848 - val_acc: 0.7308
Epoch 4/250
 - 0s - loss: 0.6746 - acc: 0.7241 - val_loss: 0.6825 - val_acc: 0.7308
Epoch 5/250
 - 0s - loss: 0.6917 - acc: 0.6034 - val_loss: 0.6800 - val_acc: 0.7308
Epoch 6/250
 - 0s - loss: 0.6746 - acc: 0.7069 - val_loss: 0.6773 - val_acc: 0.7308
Epoch 7/250
 - 0s - loss: 0.6993 - acc: 0.7241 - val_loss: 0.6750 - val_acc: 0.7308
Epoch 8/250
 - 0s - loss: 0.6914 - acc: 0.7241 - val_loss: 0.6725 - val_acc: 0.7692
Epoch 9/250
 - 0s - loss: 0.6812 - acc: 0.6724 - val_loss: 0.6699 - val_acc: 0.7692
Epoch 10/250
 - 0s - loss: 0.6753 - acc: 0.7414 - val_loss: 0.6673 - val_acc: 0.7692
Epoch 11/250
 - 0s - loss: 0.6563 - acc: 0.7414 - val_loss: 0.6644 - val_acc: 0.7692
Epoch 12/250
 - 0s - loss: 0.6

Epoch 97/250
 - 0s - loss: 0.4043 - acc: 0.7586 - val_loss: 0.4474 - val_acc: 0.7692
Epoch 98/250
 - 0s - loss: 0.3824 - acc: 0.7586 - val_loss: 0.4468 - val_acc: 0.7692
Epoch 99/250
 - 0s - loss: 0.4581 - acc: 0.7586 - val_loss: 0.4458 - val_acc: 0.7692
Epoch 100/250
 - 0s - loss: 0.4018 - acc: 0.7586 - val_loss: 0.4446 - val_acc: 0.7692
Epoch 101/250
 - 0s - loss: 0.4158 - acc: 0.7586 - val_loss: 0.4434 - val_acc: 0.7692
Epoch 102/250
 - 0s - loss: 0.3949 - acc: 0.7586 - val_loss: 0.4424 - val_acc: 0.7692
Epoch 103/250
 - 0s - loss: 0.3888 - acc: 0.7586 - val_loss: 0.4418 - val_acc: 0.7692
Epoch 104/250
 - 0s - loss: 0.3864 - acc: 0.7586 - val_loss: 0.4414 - val_acc: 0.7692
Epoch 105/250
 - 0s - loss: 0.3937 - acc: 0.7586 - val_loss: 0.4411 - val_acc: 0.7692
Epoch 106/250
 - 0s - loss: 0.4446 - acc: 0.7586 - val_loss: 0.4404 - val_acc: 0.7692
Epoch 107/250
 - 0s - loss: 0.3918 - acc: 0.7586 - val_loss: 0.4402 - val_acc: 0.7692
Epoch 108/250
 - 0s - loss: 0.3894 - acc: 0.7586 - val_lo

Epoch 193/250
 - 0s - loss: 0.2766 - acc: 0.7586 - val_loss: 0.5114 - val_acc: 0.7692
Epoch 194/250
 - 0s - loss: 0.2411 - acc: 0.7586 - val_loss: 0.5142 - val_acc: 0.7692
Epoch 195/250
 - 0s - loss: 0.2919 - acc: 0.7586 - val_loss: 0.5166 - val_acc: 0.7692
Epoch 196/250
 - 0s - loss: 0.3250 - acc: 0.7586 - val_loss: 0.5180 - val_acc: 0.7692
Epoch 197/250
 - 0s - loss: 0.3113 - acc: 0.7586 - val_loss: 0.5187 - val_acc: 0.7692
Epoch 198/250
 - 0s - loss: 0.2986 - acc: 0.7586 - val_loss: 0.5186 - val_acc: 0.7692
Epoch 199/250
 - 0s - loss: 0.2704 - acc: 0.7586 - val_loss: 0.5194 - val_acc: 0.7692
Epoch 200/250
 - 0s - loss: 0.2615 - acc: 0.7586 - val_loss: 0.5211 - val_acc: 0.7692
Epoch 201/250
 - 0s - loss: 0.2347 - acc: 0.7586 - val_loss: 0.5236 - val_acc: 0.7692
Epoch 202/250
 - 0s - loss: 0.2730 - acc: 0.7586 - val_loss: 0.5267 - val_acc: 0.7692
Epoch 203/250
 - 0s - loss: 0.2620 - acc: 0.7586 - val_loss: 0.5297 - val_acc: 0.7692
Epoch 204/250
 - 0s - loss: 0.2625 - acc: 0.7586 - val

In [18]:
def mlp_farewell(shape):
# define our MLP network
    initializer = tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=42)
    model = Sequential()
    model.add(Dense(16, input_dim=shape, kernel_initializer = initializer, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(2, activation="relu"))
# check to see if the regression node should be added
    #if regress:
    model.add(Dense(1, activation = "sigmoid"))
    #Compile model
    opt = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss='binary_crossentropy', metrics = ["accuracy"], optimizer=opt)
# return our model
    return model

In [19]:
mlp_farewell = mlp_farewell(shape)
history = mlp_farewell.fit(X_train, np.asarray(y_train["Search"]).reshape(-1,1),
                  validation_data=(X_validation, np.asarray(y_validation["Search"]).reshape(-1,1)),
    epochs=250,
    workers = 2, use_multiprocessing= True, verbose = 2)

Train on 58 samples, validate on 26 samples
Epoch 1/250
 - 1s - loss: 0.7034 - acc: 0.5862 - val_loss: 0.6971 - val_acc: 0.5769
Epoch 2/250
 - 0s - loss: 0.7015 - acc: 0.6552 - val_loss: 0.6958 - val_acc: 0.6154
Epoch 3/250
 - 0s - loss: 0.6984 - acc: 0.6724 - val_loss: 0.6946 - val_acc: 0.6538
Epoch 4/250
 - 0s - loss: 0.6963 - acc: 0.6897 - val_loss: 0.6936 - val_acc: 0.6538
Epoch 5/250
 - 0s - loss: 0.6951 - acc: 0.6552 - val_loss: 0.6927 - val_acc: 0.6538
Epoch 6/250
 - 0s - loss: 0.6860 - acc: 0.8103 - val_loss: 0.6918 - val_acc: 0.6538
Epoch 7/250
 - 0s - loss: 0.6951 - acc: 0.6897 - val_loss: 0.6908 - val_acc: 0.6538
Epoch 8/250
 - 0s - loss: 0.6961 - acc: 0.6552 - val_loss: 0.6898 - val_acc: 0.7692
Epoch 9/250
 - 0s - loss: 0.6909 - acc: 0.6897 - val_loss: 0.6888 - val_acc: 0.7692
Epoch 10/250
 - 0s - loss: 0.6890 - acc: 0.7586 - val_loss: 0.6882 - val_acc: 0.7692
Epoch 11/250
 - 0s - loss: 0.6888 - acc: 0.7241 - val_loss: 0.6877 - val_acc: 0.7692
Epoch 12/250
 - 0s - loss: 0.6

Epoch 97/250
 - 0s - loss: 0.6493 - acc: 0.7759 - val_loss: 0.6474 - val_acc: 0.7692
Epoch 98/250
 - 0s - loss: 0.6466 - acc: 0.7759 - val_loss: 0.6469 - val_acc: 0.7692
Epoch 99/250
 - 0s - loss: 0.6477 - acc: 0.7759 - val_loss: 0.6465 - val_acc: 0.7692
Epoch 100/250
 - 0s - loss: 0.6490 - acc: 0.7759 - val_loss: 0.6460 - val_acc: 0.7692
Epoch 101/250
 - 0s - loss: 0.6491 - acc: 0.7586 - val_loss: 0.6456 - val_acc: 0.7692
Epoch 102/250
 - 0s - loss: 0.6446 - acc: 0.7931 - val_loss: 0.6452 - val_acc: 0.7692
Epoch 103/250
 - 0s - loss: 0.6438 - acc: 0.7759 - val_loss: 0.6447 - val_acc: 0.7692
Epoch 104/250
 - 0s - loss: 0.6486 - acc: 0.7586 - val_loss: 0.6443 - val_acc: 0.7692
Epoch 105/250
 - 0s - loss: 0.6454 - acc: 0.7759 - val_loss: 0.6438 - val_acc: 0.7692
Epoch 106/250
 - 0s - loss: 0.6394 - acc: 0.7931 - val_loss: 0.6434 - val_acc: 0.7692
Epoch 107/250
 - 0s - loss: 0.6462 - acc: 0.7414 - val_loss: 0.6429 - val_acc: 0.7692
Epoch 108/250
 - 0s - loss: 0.6441 - acc: 0.7759 - val_lo

Epoch 193/250
 - 0s - loss: 0.5584 - acc: 0.8448 - val_loss: 0.5854 - val_acc: 0.8077
Epoch 194/250
 - 0s - loss: 0.5571 - acc: 0.8448 - val_loss: 0.5847 - val_acc: 0.8077
Epoch 195/250
 - 0s - loss: 0.5572 - acc: 0.8276 - val_loss: 0.5835 - val_acc: 0.8077
Epoch 196/250
 - 0s - loss: 0.5477 - acc: 0.8793 - val_loss: 0.5824 - val_acc: 0.8077
Epoch 197/250
 - 0s - loss: 0.5650 - acc: 0.8448 - val_loss: 0.5813 - val_acc: 0.8077
Epoch 198/250
 - 0s - loss: 0.5327 - acc: 0.8966 - val_loss: 0.5804 - val_acc: 0.8077
Epoch 199/250
 - 0s - loss: 0.5499 - acc: 0.8793 - val_loss: 0.5795 - val_acc: 0.8077
Epoch 200/250
 - 0s - loss: 0.5691 - acc: 0.8276 - val_loss: 0.5786 - val_acc: 0.8077
Epoch 201/250
 - 0s - loss: 0.5369 - acc: 0.8966 - val_loss: 0.5776 - val_acc: 0.8077
Epoch 202/250
 - 0s - loss: 0.5351 - acc: 0.8966 - val_loss: 0.5766 - val_acc: 0.8077
Epoch 203/250
 - 0s - loss: 0.5645 - acc: 0.8621 - val_loss: 0.5757 - val_acc: 0.8077
Epoch 204/250
 - 0s - loss: 0.5413 - acc: 0.8621 - val

In [20]:
mlp_greeting.save('model_greetings.h5')
mlp_search.save('model_search.h5')
mlp_suggestion.save('model_suggestion.h5')
mlp_farewell.save('model_farewell.h5')

# CHATBOT TEST

In [23]:
def chatbot():
    
    input_text = input()
    
    test = pd.DataFrame(data = {'Sentence': [input_text]})
    df_test_proc, test_proc = processing(test, cv = cv)

    gret_prob = mlp_greeting.predict_proba(test_proc)
    search_prob = mlp_search.predict_proba(test_proc)
    sugg_prob = mlp_suggestion.predict_proba(test_proc)
    
    probs = [gret_prob, search_prob, sugg_prob]
    idx = np.argmax(probs)
    
    if idx == 0:
        print("Esto es un saludo")
    elif idx == 1:
        print("Esto es una búsqueda")
    else:
        print("Esto es una sugerencia")
        
#     print('¿Hemos acertado?')
    
#     respuesta = input()
#     if (respuesta == 'No' or respuesta == 'no'):
#         probs = np.delete(probs, idx)
#         idx_2 = np.argmax(probs)
        
#         if idx == 0:
#             if idx_2 == 0:
#                 print("Esto es una búsqueda")
#             else:
#                 print("Esto es una sugerencia")
#         elif idx == 1:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una sugerencia")
#         else:
#             if idx_2 == 0:
#                 print("Esto es un saludo")
#             else:
#                 print("Esto es una búsqueda")
#     else:
#         print('¡Genial! ¡Hemos acertado!')
    

In [35]:
chatbot()

Tell me things related to Napoleon
Esto es un saludo


* Correctamente
 * La mayoría de saludos
 * I want to know about Data Science (búsqueda)
 * I want to know things connected to la Sagrada Familia (sugerencia)
 * Watcha doing'? (saludo)
 * Tell me things related to Data Science (sugerencia)

* Incorrectamente
 * What is Data Science (sugerencia)
 * Tell me things related to Napoleon (saludo)
 * Who is Cristobal Colon (saludo)



* Opciones
 * Mostrar un comando *!options* que te de las opciones para hacer.
 * Detectar un intent
 * Mostrarlo al inicio


* Headers
 * Dar las opciones una vez se detecta intent de búsqueda búsqueda
   * *What is data science?* Y mostrarle los heades para qué elija qué queire saber 
 * Intent nuevo de header